# 02 — Exploratory Data Analysis (Dataset Audit)


## Purpose
This notebook audits the IEEE-CIS fraud dataset to understand:
- table structure and join keys
- target definition and class imbalance
- missingness patterns
- time semantics and leakage risks

## Rules
- No modeling in this notebook
- No feature engineering in this notebook
- No test set usage except schema comparison if needed



In [21]:
import sys
print(sys.executable)


c:\Projects\fraud-detection-ml\venv\Scripts\python.exe


In [22]:
import numpy as np
import pandas as pd

# Display settings for audit-style EDA
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)
pd.set_option("display.float_format", "{:.4f}".format)


In [23]:
from pathlib import Path

DATA_DIR = Path("..") / "data" / "raw"   
TRAIN_TX_PATH = DATA_DIR / "train_transaction.csv"
TRAIN_ID_PATH = DATA_DIR / "train_identity.csv"

print("Data dir:", DATA_DIR.resolve())
print("Train transaction exists:", TRAIN_TX_PATH.exists())
print("Train identity exists:", TRAIN_ID_PATH.exists())


Data dir: C:\Projects\fraud-detection-ml\data\raw
Train transaction exists: True
Train identity exists: True


In [24]:
train_tx = pd.read_csv(TRAIN_TX_PATH)
train_id = pd.read_csv(TRAIN_ID_PATH)

print("train_transaction shape:", train_tx.shape)
print("train_identity shape:", train_id.shape)


train_transaction shape: (590540, 394)
train_identity shape: (144233, 41)


In [25]:
train_tx.info()


<class 'pandas.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 394 entries, TransactionID to V339
dtypes: float64(376), int64(4), str(14)
memory usage: 1.7 GB


In [26]:
train_id.info()


<class 'pandas.DataFrame'>
RangeIndex: 144233 entries, 0 to 144232
Data columns (total 41 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionID  144233 non-null  int64  
 1   id_01          144233 non-null  float64
 2   id_02          140872 non-null  float64
 3   id_03          66324 non-null   float64
 4   id_04          66324 non-null   float64
 5   id_05          136865 non-null  float64
 6   id_06          136865 non-null  float64
 7   id_07          5155 non-null    float64
 8   id_08          5155 non-null    float64
 9   id_09          74926 non-null   float64
 10  id_10          74926 non-null   float64
 11  id_11          140978 non-null  float64
 12  id_12          144233 non-null  str    
 13  id_13          127320 non-null  float64
 14  id_14          80044 non-null   float64
 15  id_15          140985 non-null  str    
 16  id_16          129340 non-null  str    
 17  id_17          139369 non-null  float64


In [27]:
common_cols = set(train_tx.columns).intersection(set(train_id.columns))
common_cols


{'TransactionID'}

In [28]:
"isFraud" in train_tx.columns


True

In [29]:
train_tx["isFraud"].value_counts()


isFraud
0    569877
1     20663
Name: count, dtype: int64

In [30]:
tx_missing = train_tx.isna().mean().sort_values(ascending=False)
tx_missing.head(10)


dist2   0.9363
D7      0.9341
D13     0.8951
D14     0.8947
D12     0.8904
D6      0.8761
D9      0.8731
D8      0.8731
V153    0.8612
V149    0.8612
dtype: float64

In [31]:
id_missing = train_id.isna().mean().sort_values(ascending=False)
id_missing.head(10)


id_24   0.9671
id_25   0.9644
id_07   0.9643
id_08   0.9643
id_21   0.9642
id_26   0.9642
id_23   0.9642
id_27   0.9642
id_22   0.9642
id_18   0.6872
dtype: float64

In [32]:
col = tx_missing.index[0]

train_tx.assign(
    is_missing=train_tx[col].isna()
).groupby(["is_missing", "isFraud"]).size()


C:\Users\Mambo\AppData\Local\Temp\ipykernel_7056\220215209.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_tx.assign(


is_missing  isFraud
False       0           33896
            1            3731
True        0          535981
            1           16932
dtype: int64

In [33]:
train_tx["TransactionDT"].describe()


count     590540.0000
mean     7372311.3101
std      4617223.6465
min        86400.0000
25%      3027057.7500
50%      7306527.5000
75%     11246620.0000
max     15811131.0000
Name: TransactionDT, dtype: float64

In [34]:
(train_tx["TransactionDT"].diff().dropna() >= 0).all()


np.True_

In [35]:
train_tx.assign(
    time_bin=pd.qcut(train_tx["TransactionDT"], q=10)
).groupby("time_bin")["isFraud"].mean()


C:\Users\Mambo\AppData\Local\Temp\ipykernel_7056\1816576924.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_tx.assign(


time_bin
(86399.999, 1361004.4]     0.0276
(1361004.4, 2310159.6]     0.0202
(2310159.6, 3864163.9]     0.0373
(3864163.9, 5592303.6]     0.0429
(5592303.6, 7306527.5]     0.0396
(7306527.5, 8745782.4]     0.0355
(8745782.4, 10437998.1]    0.0432
(10437998.1, 12192853.6]   0.0349
(12192853.6, 13990907.7]   0.0313
(13990907.7, 15811131.0]   0.0375
Name: isFraud, dtype: float64

In [36]:
train_merged = train_tx.merge(
    train_id,
    on="TransactionID",
    how="left"
)

print("Transactions before merge:", train_tx.shape)
print("After merge:", train_merged.shape)


Transactions before merge: (590540, 394)
After merge: (590540, 434)


In [37]:
identity_cols = train_id.columns.drop("TransactionID")

train_merged[identity_cols].isna().all(axis=1).value_counts()


True     446307
False    144233
Name: count, dtype: int64

## Feature Groups (Initial Definition)

### Core Transaction Features
These features are always present for every transaction and form the backbone of the fraud detection model.  
They must be sufficient for the model to make a reasonable prediction even when no additional context is available.

### Identity Features (Optional Context)
These features provide additional context about the device, browser, or identity behind a transaction.  
They are often missing and must improve the model when present, but the model must not depend on them.

### Anonymized Engineered Features
These features are anonymized or abstract representations of behavioral signals.  
They may be highly predictive but lack direct semantic meaning and should be interpreted at a group level rather than individually.

### Time-Derived Features
These features are derived from transaction timing and historical behavior.  
They must strictly respect temporal order to avoid data leakage and are critical for modeling fraud dynamics over time.
